# Gravitational Search Algorithm para Problema de la Mochila Paralelizado

## Importamos las librerías necesarias

In [17]:
import math
import numpy as np
import pandas as pd
from time import time
from numba import cuda
import cupy as cp
import random

## Definimos las funciones a utilizar

In [18]:
#Función para calcular el peso de una solucion
def calcularPeso(sol,values):
    peso = 0
    for i in range(N):
        peso = peso + sol[i]*(values[i][0])
    return peso

In [19]:
#Funcion para calcular el fitness de la poblacion
@cuda.jit
def calcularFitness(population,values,fitness):
    i,j= cuda.grid(2)  
    if i < population.shape[0] and j < 1:
        valor = 0
        for k in range(population.shape[1]):
            valor = valor + population[i,k]*(values[k,1])
        fitness[i]=valor

In [20]:
#Funcion para generar de forma aleatoria los agentes iniciales
def initialPopulation(values,N,P,W):
    population = []
    iteraciones = N
    for i in range(P):
        sol = [0]*N
        peso = 0  
        j = 0
        while peso <= W and j < iteraciones:
            pos = random.randrange(0,N,1)
            sol[pos] = 1
            pesoAnt = peso
            peso = calcularPeso(sol,values)
            if peso > W:
                sol[pos] = 0
                peso = pesoAnt
            j=j+1
        population.append(sol)
    return population


In [21]:
#Funcion de actualizacion de la constante de gravedad
def actualizarG(t,G0,alpha,maxIter):
    G = (-alpha * t)/maxIter
    G = math.exp(G)
    G = G * G0
    return G

In [22]:
#Funcion de calculo de las masas de cada agente
def calcularMasas(fit,best,worst,P):
    m=[]
    M=[]
    for i in range(P):
        m.append((fitness[i]-worst)/(best-worst))
    for i in range(P):
        suma = 0
        for j in m:
            suma = suma + j   
        M.append(m[i]/suma)
    return M

In [23]:
#Funcion de resta de posicion especifica
def restPos(xi,xj,d):
    return xj[d] - xi[d]

In [24]:
#Funcion de calculo de la distancia euclidiana
def calcularR(x1,x2,N):
    suma=0
    for i in range(N):
        suma = suma + (x2[i]-x1[i])**2
    R=math.sqrt(suma)
    return R

In [25]:
#Funcion de calculo de las fuerzas en todas las direcciones Fijd y la fuerza total aplicada en cada agente en cada direccion Fid
def calcularFuerzas(P,N,G,M,population,e):
    FT=[]
    Ft=[]
    f=[]
    
    for i in range(P):
        Ft=[]
        for j in range(P):
            f=[]
            for d in range(N):
                value=0
                value = value+M[i]*M[j]
                value = value/(calcularR(population[i],population[j],N)+e)
                value = value*G*restPos(population[i],population[j],d)
                f.append(value)
            Ft.append(f)
        FT.append(Ft)
    F=[]
    f=[]
    for i in range(P):
        f=[]
        for d in range(N): 
            value=0
            for j in range(P):
                if j != i :
                    r = random.random()
                    value = value +(r*np.array(FT)[i,j,d])
            f.append(value)
        F.append(f)
    return(F)

In [26]:
#Funcion de actualizacion de aceleraciones
@cuda.jit
def calcularAceleracion(M,F,a):
    i,j= cuda.grid(2)
    if i < M.shape[0] and j < F.shape[1]:
        if F[i,j] > 0 and M[i] > 0:
            a[i,j] = F[i,j]/M[i]
        else:
            a[i,j] = 0

In [27]:
#Funcion de actualizacion de velocidades
@cuda.jit
def calcularVelocidad(v,a):
    i,j= cuda.grid(2)
    if i<a.shape[0] and j<a.shape[1]:
        value = (v[i,j]+a[i,j])
        v[i,j] = value
            

In [28]:
#Funcion de actualizacion de posiciones
def calcularPosiciones(P,N,population,v,values,W):
    for i in range(P):
        for j in range(N):
            if v[i][j] > 0 and population[i][j]==0:
                population[i][j]= 1
                if calcularPeso(population[i],values) > W:
                    population[i][j]= 0
            if v[i][j] < 0 and population[i][j]==1:
                population[i][j]= 0
    return population

In [29]:
#Funcion de lectura del problema en los csv
def leerCSV():
    problema2 = pd.read_csv('p.csv')
    param = pd.read_csv('s.csv')
    lista = []
    for i in range(problema2['Peso'].size):
        item = [problema2['Peso'][i],problema2['Valor'][i]]
        lista.append(item)
    N = param['Parametros'][0]
    P = param['Parametros'][1]
    W = param['Parametros'][2]
    return N,P,W,lista

## Inicializamos Variables

In [30]:
#Leemos el problema
[N,P,W,values] = leerCSV()
#Inicializamos poblacion
population = initialPopulation(values,N,P,W)
#Inicializamos parametros
G0 = 100.0
v=list(np.zeros((P, N)))
best = []
worst = []
M=[]
F = []
a=[]
e=0.5
maxIter = (N//2)+ P
alpha = maxIter*0.2
print('N: ',N)
print('P: ',P)
print('W: ',W)
print("Poblacion: ",population)




N:  16
P:  30
W:  31
Poblacion:  [[0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1], [1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1], [0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1], [1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1], [1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1], [0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1], [0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1], [1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1], [0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1], [0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0], [0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0], [1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1], [0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0], [0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1], [0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0], [0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0], [1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0], [1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1], [1, 0, 0, 0, 0, 

## Ciclo del algoritmo GSA

In [31]:
%%time
#INICIO
#Calculamos fitness
gpu_Population = cp.asarray(population)
gpu_Values = cp.asarray(values)
threadsperblock = (20, 20)  # each block will contain 16x16 threads, typically 128 - 512 threads/block
blockspergrid_x = int(np.ceil(gpu_Population.shape[0] / threadsperblock[0]))
blockspergrid_y = int(np.ceil(gpu_Population.shape[1] / threadsperblock[1]))
blockspergrid = (blockspergrid_x, blockspergrid_y)  # we calculate the gridsize (number of blocks) from array
for i in range(maxIter):
    #Calculamos fitness
    gpu_Population = cp.asarray(population)
    gpu_Values = cp.asarray(values)
    # execution of the kernel
    fit = cp.zeros((len(population)), dtype=np.int32) 
    calcularFitness[blockspergrid, threadsperblock](gpu_Population,gpu_Values,fit)
    fitness= fit.tolist()
    # print("fitness: ",fitness)
    #Actualizamos G,best y worst
    G = actualizarG(i,G0,alpha,maxIter)
    # print("G: ",G)
    b = max(fitness)
    best = fitness.index(b)
    w = min(fitness)
    worst = fitness.index(w)
    # print('Best: ' + str(population[best]) + str(fitness[best]))
    # print('Worst: '+ str(population[worst]) + str(fitness[worst]))
    #Imprimimos la media y la desviacion estandar de los fitness
    # print('Media de Fitness: ',np.mean(fitness))
    # print('Desviacion estandar: ',np.std(fitness))
    #Actualizamos Masas(M), Fuerzas(Fid), aceleraciones(aid) y velocidades(vid)
    M = calcularMasas(fitness,b,w,P)
    # print("Masas: ",M)
    F = calcularFuerzas(P,N,G,M,population,e)
    # print("Fuerzas sobre cada agente: ")
    # print(np.array(F))
    
    gpu_a = cp.zeros((P, N), dtype=np.float32)
    gpu_M = cp.asarray(M)
    gpu_F = cp.asarray(F)
    
   
    # execution of the kernel
    calcularAceleracion[blockspergrid, threadsperblock](gpu_M,gpu_F,gpu_a)
    a= gpu_a.tolist()
    #print("Aceleracion: ",a)
    
    gpu_v = cp.zeros((P, N), dtype=np.float32)
    
    calcularVelocidad[blockspergrid, threadsperblock](gpu_v,gpu_a)
    v = gpu_v.tolist()
    #print("Velocidad: ",v)
    #Actualizamos posiciones (xid)
    population = calcularPosiciones(P,N,population,v,values,W)
    #Mostramos Nueva Poblacion
    # print("-------------------------------------------------------------------")
    # print("Nueva Poblacion: ",population)

#FIN

Wall time: 10min 17s


## Imprimimos resultados

In [32]:
print('Best: ' + str(population[best]) + str(fitness[best]))
print('Worst: '+ str(population[worst]) + str(fitness[worst]))

Best: [1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1]78
Worst: [0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0]26
